<a href="https://colab.research.google.com/github/Deepandreinforcement/Python-Create-Own-Image/blob/main/AutoTrain_Dreambooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Eğitme

Burada ilk önce kendi resimlerimizi kullanarak bir eğitim işlemi yapacağız. Eğitim işlemi bittikten sonra bu oluşan ağırlıkları kullanarak kendi resimlerimizi oluşturabiliriz

Bu kod ile images adlı bir klasör oluşturuyoruz. Bu aşağıdaki kodu çalıştırdıktan sonra oluşan images klasörünü kendimize ait resimleri yüklüyoruz. Burada en az 10-15 adet yüzümüzün net çıktığı resimler olmalı.

In [ ]:
import os
os.mkdir("images")

Burada eğitim işlemi için gerekli kurulumları yapıyoruz



In [ ]:
#@title 🤗 AutoTrain DreamBooth
#@markdown In order to use this colab
#@markdown - upload images to a folder named `images/`
#@markdown - choose a project name if you wish
#@markdown - change model if you wish, you can also select sd2/2.1 or sd1.5
#@markdown - update prompt and remember it. choose keywords that don't usually appear in dictionaries
#@markdown - add huggingface information (token and repo_id) if you wish to push trained model to huggingface hub
#@markdown - update hyperparameters if you wish
#@markdown - click `Runtime > Run all` or run each cell individually
#@markdown - report issues / feature requests here: https://github.com/huggingface/autotrain-advanced/issues

import os
!pip install -U autotrain-advanced > install_logs.txt

Burada eğitim için bazı ayarlamaları yapıyoruz. Bu şekilde kalabilir çok bir değişiklik yapılmasına gerek yok

In [ ]:
#@markdown ---
#@markdown #### Project Config
project_name = 'my_dreambooth_project' # @param {type:"string"}
model_name = 'stabilityai/stable-diffusion-xl-base-1.0' # @param ["stabilityai/stable-diffusion-xl-base-1.0", "runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-2-1", "stabilityai/stable-diffusion-2-1-base"]
prompt = 'photo of a sks person' # @param {type: "string"}

#@markdown ---
#@markdown #### Push to Hub?
#@markdown Use these only if you want to push your trained model to a private repo in your Hugging Face Account
#@markdown If you dont use these, the model will be saved in Google Colab and you are required to download it manually.
#@markdown Please enter your Hugging Face write token. The trained model will be saved to your Hugging Face account.
#@markdown You can find your token here: https://huggingface.co/settings/tokens
push_to_hub = False # @param ["False", "True"] {type:"raw"}
hf_token = "hf_XXX" #@param {type:"string"}
repo_id = "username/repo_name" #@param {type:"string"}

#@markdown ---
#@markdown #### Hyperparameters
learning_rate = 1e-4 # @param {type:"number"}
num_steps = 500 #@param {type:"number"}
batch_size = 1 # @param {type:"slider", min:1, max:32, step:1}
gradient_accumulation = 4 # @param {type:"slider", min:1, max:32, step:1}
resolution = 1024 # @param {type:"slider", min:128, max:1024, step:128}
use_8bit_adam = False # @param ["False", "True"] {type:"raw"}
use_xformers = False # @param ["False", "True"] {type:"raw"}
mixed_precision = "fp16" # @param ["fp16", "bf16", "none"] {type:"raw"}
train_text_encoder = False # @param ["False", "True"] {type:"raw"}
disable_gradient_checkpointing = False # @param ["False", "True"] {type:"raw"}

os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PROMPT"] = prompt
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
os.environ["REPO_ID"] = repo_id
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)
os.environ["USE_8BIT_ADAM"] = str(use_8bit_adam)
os.environ["USE_XFORMERS"] = str(use_xformers)
os.environ["MIXED_PRECISION"] = str(mixed_precision)
os.environ["TRAIN_TEXT_ENCODER"] = str(train_text_encoder)
os.environ["DISABLE_GRADIENT_CHECKPOINTING"] = str(disable_gradient_checkpointing)

Burada artık modeli eğitiyoruz. Bu işlem biraz uzun sürer. Eğitim sonucunda my_dreambooth_project klasörünün altında pytorch_lora_weights.safetensors adlı bir dosya oluşacak. Bu dosyayı indirebilirsiniz. Böylece artık eğitim işlemine gerek kalmadan bu dosyayı yükleyerek direk görsellerinizi üretebilirsiniz.

In [ ]:
!autotrain dreambooth \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--image-path images/ \
--prompt "${PROMPT}" \
--resolution ${RESOLUTION} \
--batch-size ${BATCH_SIZE} \
--num-steps ${NUM_STEPS} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--lr ${LEARNING_RATE} \
--mixed-precision ${MIXED_PRECISION} \
$( [[ "$USE_XFORMERS" == "True" ]] && echo "--xformers" ) \
$( [[ "$TRAIN_TEXT_ENCODER" == "True" ]] && echo "--train-text-encoder" ) \
$( [[ "$USE_8BIT_ADAM" == "True" ]] && echo "--use-8bit-adam" ) \
$( [[ "$DISABLE_GRADIENT_CHECKPOINTING" == "True" ]] && echo "--disable_gradient-checkpointing" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}" )

# Önceden Eğitilmiş Modeli Kullanma

Eğer daha önce burada bu şekilde bir model eğittiyseniz yeniden yukarıdaki model eğitme adımını yapmanıza gerek yok. Fakat ilk defa yapıyorsanız da alttaki kodu çalıştırmayın. Çünkü yukarıdaki işlemler esnasında zaten buradaki kodun yaptığı işlem yapılıyor.

Burada my_dreambooth_project adlı bir klasör oluşturuyoruz. Bu kod çalıştıktan sonra önceden eğitilmiş ağırlıkları buraya yüklüyoruz.


In [ ]:
import os
os.mkdir("my_dreambooth_project")

# Modeli Tanımlama



Burada eğittiğimiz ya da yüklediğimiz ağırlıkları kullanarak modeli oluşturyoruz.

In [ ]:
from diffusers import DiffusionPipeline
import torch

prj_path = "my_dreambooth_project"
model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
     model,
     torch_dtype=torch.float16,
 )
pipe.to("cuda")
pipe.load_lora_weights(prj_path, weight_name="pytorch_lora_weights.safetensors")

# Görsel Oluşturma

Burada nasıl bir görsel oluşturacağımız prompt ile belirtiyoruz. Ben burada birçok prompt örneği koydum. İstediğinizi yorumdan çıkararak deneyebilirsiniz.

In [ ]:
prompt = "photo of a sks person as a doctor, hyper realistic, comic book style color, "

# prompt = "drawing  of a sks person  pencil drawing on white paper "
# prompt = "wall painting   of a sks person on wall in New York Streets  , ultra realistic"
# prompt = "photo of a sks person as  astronaut , ultra realistic, ultra high quality  "
# prompt = "comic book drawing of a sks person, ultra realistic, comic book style color,  "
# prompt = "graffiti art  of a sks person, ultra realistic, comic book style color "
# prompt = "billboard on street of a sks person in New York street  , hyper realistic comic book style style color "
# prompt = "photo  of a sks person as on picture tableau , colurful, hyper realistic , comic book style color "

negative_prompt  resimde olmasını istemediğimiz şeyleri belirtiyor.

In [ ]:
negative_prompt='worst quality, poor details, ugly faces, blury faces,',


Burada görseli oluşturyoruz. Burada seed'i manuel olarak biz atıyoruz. Her bir seed için farklı bir görsel oluşturuluyor. Seed'i değiştirerek daha iyi sonuçlar elde edebilirsiniz. Burada num_inference_steps resmin kaç aşamada oluşturulacağını belirtiyor. negative_prompt ise resimde olmasını istemediğimiz şeyleri belirtiyor.

In [ ]:
seed = 256
generator = torch.Generator("cuda").manual_seed(seed)
image = pipe(prompt=prompt,

             #num_inference_steps=50,
             negative_prompt=negative_prompt,
             generator=generator
             ).images[0]

  0%|          | 0/50 [00:00<?, ?it/s]

İsterseniz seed'i manuel olarak belirlemek yerine rastgele bir şekilde belirleyebilirsiniz. Bu kodu her çalıştırdığınızda farklı bir görsel üretilecek.

In [ ]:
image = pipe(prompt=prompt,
             negative_prompt=negative_prompt
             ).images[0]

Bazı prompt ve negative_prompt örnekleri

In [ ]:
prompt = "steampunk style of a sks person antique, mechanical, brass and copper tones, gears, intricate, detailed"
negative_prompt='deformed, glitch, noisy, low contrast, anime, photorealistic'

prompt = "GTA-style artwork of a sks person  satirical, exaggerated, pop art style, vibrant colors, iconic characters, action-packed  "
negative_prompt='realistic, black and white, low contrast, impressionist, cubist, noisy, blurry, deformed'

prompt = "low-poly style of a sks person low-poly game art, polygon mesh, jagged, blocky, wireframe edges, centered composition"
negative_prompt='noisy, sloppy, messy, grainy, highly detailed, ultra textured, photo'

prompt = "film noir style  photo of a sks person monochrome, high contrast, dramatic shadows, 1940s style, mysterious, cinematic"
negative_prompt='ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, vibrant, colorful'

prompt = "renaissance style of a sks person realistic, perspective, light and shadow, religious or mythological themes, highly detailed"
negative_prompt='ugly, deformed, noisy, blurry, low contrast, modernist, minimalist, abstract'

prompt = "paper quilling art of of a sks person  intricate, delicate, curling, rolling, shaping, coiling, loops, 3D, dimensional, ornamental"
negative_prompt='photo, painting, drawing, 2D, flat, deformed, noisy, blurry'

prompt = "abstract style of a sks person non-representational, colors and shapes, expression of feelings, imaginative, highly detailed"
negative_prompt='realistic, photographic, figurative, concrete'

prompt = "pop Art style of a sks person bright colors, bold outlines, popular culture themes, ironic or kitsch"
negative_prompt='ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, minimalist'

prompt = "analog film photo of a sks person faded film, desaturated, 35mm photo, grainy, vignette, vintage, Kodachrome, Lomography, stained, highly detailed, found footage"
negative_prompt='painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured'


prompt = "stained glass style of a sks person, vibrant, beautiful, translucent, intricate, detailed"
negative_prompt='ugly, deformed, noisy, blurry, low contrast, realism, photorealistic'




Görseli ekranda gösteriyoruz.

In [ ]:
image

Görseli kaydediyoruz.

In [ ]:
image.save(f"saved_image.png")